# Simple transformers classification



In [0]:
import pandas as pd
import numpy as np
import logging

pd.options.display.max_columns=None

In [2]:
%%writefile setup.sh

export CUDA_HOME=/usr/local/cuda-10.1
git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

Overwriting setup.sh


In [3]:
!sh setup.sh

fatal: destination path 'apex' already exists and is not an empty directory.
/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-xl640ude
Created temporary directory: /tmp/pip-req-tracker-6k14koa7
Created requirements tracker '/tmp/pip-req-tracker-6k14koa7'
Created temporary directory: /tmp/pip-install-w_kt_ur5
Processing ./apex
  Created temporary directory: /tmp/pip-req-build-6zzpp7me
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker-6k14koa7'
    Running setup.py (path:/tmp/pip-req-build-6zzpp7me/setup.py) egg_info for package from file:///content/apex
    Running command python setup.py egg_info
    torch.__version__  =  1.5.0+cu101
    running egg_info
    creating /tmp/pip-req-build-6zzpp7me/pip-egg-info/apex.egg-info
  

In [4]:
!pip install simpletransformers

In [0]:
#!pip install -v --no-cache-dir apex/

In [6]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [0]:
from simpletransformers.classification import ClassificationModel

We want to use simpletransformers to implement the huggingface transformer methods

In [0]:
#from google.colab import drive
#drive.mount('/content/drive')

In [9]:
# fantasy paranormal books and reviews, along with gathered goodreads 
# hugo/nebula/locus nominees

nrows = 5000
bks = pd.read_csv('/content/drive/My Drive/data/booksFPawardsReviewsFPall.csv',
                  nrows=nrows,usecols=[1,3,22,23,33,34,35])
print(bks.head())

         isbn        asin  rating  \
0         NaN  B00071IKUY       3   
1         NaN  B00071IKUY       4   
2         NaN  B00071IKUY       3   
3  1934876569         NaN       3   
4         NaN  B01NCIKAQX       3   

                                         review_text award  Nominee  \
0  Competent. I'm sure I read the first half year...   NaN      NaN   
1  While these books are excellent overall, I thi...   NaN      NaN   
2  A collection of two novels about the mercenary...   NaN      NaN   
3  This was a really cute book, though to be hone...   NaN      NaN   
4  Lots and lots of filler. I was skimming toward...   NaN      NaN   

   Nominee_Winner  
0             NaN  
1             NaN  
2             NaN  
3             NaN  
4             NaN  


In [10]:
#display(bks.head())
bks[['Nominee','Nominee_Winner']]=bks[['Nominee','Nominee_Winner']].fillna(0).astype(int)
bks.dtypes

isbn              object
asin              object
rating             int64
review_text       object
award             object
Nominee            int64
Nominee_Winner     int64
dtype: object

We want a feature that is a 1 if there exists a matching entry in the awards data. 

In [11]:
bks['awardNominee'] = bks['award'].notna().astype(int)
bksClass = bks[['isbn','asin','review_text','awardNominee']].rename(columns={'review_text':'text','awardNominee':'labels'})
bksClass = bksClass[bksClass['text'].notna()]
print(bksClass.head())

         isbn        asin                                               text  \
0         NaN  B00071IKUY  Competent. I'm sure I read the first half year...   
1         NaN  B00071IKUY  While these books are excellent overall, I thi...   
2         NaN  B00071IKUY  A collection of two novels about the mercenary...   
3  1934876569         NaN  This was a really cute book, though to be hone...   
4         NaN  B01NCIKAQX  Lots and lots of filler. I was skimming toward...   

   labels  
0       0  
1       0  
2       0  
3       0  
4       0  


## Minimal start 

from https://simpletransformers.ai/docs/binary-classification/


In [12]:
randomSelection = np.random.rand(len(bksClass)) < 0.8
train_data = bksClass[randomSelection][['text','labels']]
test_data =  bksClass[~randomSelection][['text','labels']]
print('train {} rows'.format(len(train_data)))
print('test {} rows'.format(len(test_data)))

train 4002 rows
test 997 rows


In [13]:
train_data.dtypes

text      object
labels     int64
dtype: object

In [14]:
# Optional model configuration
model_args = {
    "num_train_epochs": 1,
    "sliding_window": True
}

# Create a ClassificationModel
model = ClassificationModel(
    "roberta", "roberta-base", args=model_args
)

# Train the model
model.train_model(train_data)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(test_data)



Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


Running loss: 0.570451

/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)
/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:114: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Running loss: 1.494738Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Running loss: 0.618157

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Running loss: 1.275192Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
Running loss: 0.527655Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8192.0
Running loss: 0.001906


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


In [25]:
print(result)
acc = (result['tp']+result['tn'])/(result['tp']+result['fp']+result['tn']+result['fn'])
print('Test accuracy is {}'.format(acc))

{'mcc': 0.7276455662889472, 'tp': 33, 'tn': 938, 'fp': 2, 'fn': 24, 'eval_loss': 0.10529733989905152}
Test accuracy is 0.9739217652958877


# Predicting bigger data

reload but instead of 5,000 records let's go up to 50,000 using the model we just tuned.

## Load saved model

This is only needed if you want to evaluate without training. You can use a saved model from the 'outputs' folder that the train step uses. If you still have model loaded in memory you shouldn't run this.

In [0]:
model_args = {
    "num_train_epochs": 1,
    "sliding_window": True
}

# Create a ClassificationModel
model = ClassificationModel(
    "roberta", "/content/drive/My Drive/data/outputs", args=model_args
)


## Load more data

This is another version of the previous code to load from the csv

In [13]:
# fantasy paranormal books and reviews, along with gathered goodreads 
# hugo/nebula/locus nominees

nrows = 50000
bks2 = pd.read_csv('/content/drive/My Drive/data/booksFPawardsReviewsFPall.csv',
                  nrows=nrows,usecols=[1,3,22,23,33,34,35])

bks2[['Nominee','Nominee_Winner']]=bks2[['Nominee','Nominee_Winner']].fillna(0).astype(int)
bks2['awardNominee'] = bks2['award'].notna().astype(int)
bks2Class = bks2[['isbn','asin','review_text','awardNominee']].rename(columns={'review_text':'text','awardNominee':'labels'})
bks2Class = bks2Class[bks2Class['text'].notna()]
print(len(bks2Class))


49971


## Evaluate model on the bigger data

In [14]:
result2, model_outputs2, wrong_predictions2 = model.eval_model(bks2Class[['text','labels']])
print(result2)
acc2 = (result2['tp']+result2['tn'])/(result2['tp']+result2['fp']+result2['tn']+result2['fn'])
print('Test accuracy is {}'.format(acc2))


{'mcc': 0.1604385340013124, 'tp': 348, 'tn': 42039, 'fp': 107, 'fn': 7477, 'eval_loss': 0.7173741737928293}
Test accuracy is 0.8482319745452362
